## Training Module

In [1]:
import os
os.chdir('genre_classification_289a/src')

In [2]:
import torch
import torch.nn as nn
import os
from model import STN, MLP
import torch.optim as optim

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
from features import get_data_loaders, genre_counts, FramedFeatureDataset, FeatureDataset

Rock                   6911
Electronic             6110
Experimental           2207
Hip-Hop                2109
Folk                   1477
Instrumental           1280
Pop                    1129
International          1004
Classical               598
Old-Time / Historic     510
Jazz                    380
Country                 178
Soul-RnB                154
Spoken                  118
Blues                    72
Easy Listening           21
Name: genre_top, dtype: int64
{'Rock': 0, 'Electronic': 1, 'Experimental': 2, 'Hip-Hop': 3, 'Folk': 4, 'Instrumental': 5, 'Pop': 6, 'International': 7, 'Classical': 8, 'Old-Time / Historic': 9, 'Jazz': 10, 'Country': 11, 'Soul-RnB': 12, 'Spoken': 13, 'Blues': 14, 'Easy Listening': 15}


## DCNN Training

In [5]:
num_genres = len(genre_counts)

In [6]:
#STN targets
agfs = [] #'subgenre', 'mfcc'
genre = True #False if not genre STN

#dataset
dataset_name = 'fma_medium'

In [7]:
output_size = num_genres
stn = STN(output_size)
stn.to(device)
stn = nn.DataParallel(stn)

In [8]:
## Training Parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(stn.parameters(), lr=0.001)
epochs = 30
batch_size = 64
valid_split = 0.2

In [9]:
dataset = FramedFeatureDataset(agfs=agfs, genre=genre)

In [10]:
trainloader, validloader = get_data_loaders(dataset, batch_size, valid_split)

In [11]:
import torch.nn.functional as F
from sklearn.metrics import f1_score

def validate(stn, label_name):
    
    with torch.no_grad():
        stn.eval()
        
        all_pred = []
        all_true = []
        
        for i, data in enumerate(validloader, 0):
            inputs, labels = data[0].to(device), data[1][label_name].to(device)
            
            out = stn(inputs)
            loss = F.cross_entropy(out, labels)
            
            all_pred.append(out.argmax(dim=1))
            all_true.append(labels)
            
        all_pred = torch.cat(all_pred)
        all_true = torch.cat(all_true)
        
        curr_f1 = f1_score(all_true.cpu(), all_pred.cpu(), average='micro')
        
        print('Validation f1 score: {}'.format(curr_f1))

In [ ]:
#Train it
for epoch in range(epochs):  # loop over the dataset multiple times
    
    print('Starting epoch', epoch + 1)
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data[0].to(device), data[1]['genre'].to(device)  #data[1]['{argument for agf being trained}']
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = stn(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 30 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0
            
    if epoch % 5 == 4:
        validate(stn, 'genre')
        stn.train()

print('Finished Training')

Starting epoch 1
[1,    50] loss: 3.491
[1,   100] loss: 2.868
[1,   150] loss: 2.601
[1,  1900] loss: 1.940
[1,  1950] loss: 1.942
[1,  2000] loss: 1.918
[1,  2050] loss: 1.944
[1,  2100] loss: 1.902
Starting epoch 2
[2,    50] loss: 1.813
[2,   100] loss: 1.836
[2,   150] loss: 1.926
[2,   200] loss: 1.839
[2,   250] loss: 1.898
[2,   300] loss: 1.817
[2,   350] loss: 1.809
[2,   400] loss: 1.839
[2,   450] loss: 1.875
[2,   500] loss: 1.834
[2,   550] loss: 1.883
[2,   600] loss: 1.855
[2,   650] loss: 1.829
[2,   700] loss: 1.835
[2,   750] loss: 1.834
[2,   800] loss: 1.756
[2,   850] loss: 1.771
[2,   900] loss: 1.781
[2,   950] loss: 1.813
[2,  1000] loss: 1.804
[2,  1050] loss: 1.850
[2,  1100] loss: 1.808
[2,  1150] loss: 1.817
[2,  1200] loss: 1.789
[2,  1250] loss: 1.795
[2,  1300] loss: 1.786
[2,  1350] loss: 1.820
[2,  1400] loss: 1.837
[2,  1450] loss: 1.772
[2,  1500] loss: 1.751
[2,  1550] loss: 1.722
[2,  1600] loss: 1.748
[2,  1650] loss: 1.730
[2,  1700] loss: 1.738


## Load Module

In [ ]:
model_file = '../models/DCNN_{}_{}'.format(dataset_name, 'genre') #switch 'genre' with agf being trained
torch.save(stn, model_file)

In [ ]:
model_file = '../models/DCNN_{}_{}'.format(dataset_name, 'genre') #switch 'genre' with agf being trained
model = torch.load(model_file)
model

In [ ]:
validate(model, 'genre') #switch 'genre' with agf being trained